<img width="8%" alt="Plotly.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Plotly.png" style="border-radius: 15%">

# Plotly - Create analytics

**Tags:** #plotly #html #csv #image #content #analytics #dependency

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook creates a chart to follow your analytics.

## Input

### Import libraries

In [ ]:
import plotly.graph_objects as go
from naas_drivers import gsheet
import pandas as pd
import os
from datetime import date, datetime, timedelta
import naas_data_product

### Setup variables
**Inputs**
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.
- `linkedin_url`: This variable stores the LinkedIn company URL that will be used as an input for the script.
- `title`: Graph title
- `col_date`: Date col to be used as x axis
- `col_value`: Value col to be aggregated on dataframe
- `agg_value`: Type of aggregation to perform on value col

**Outputs**
- `output_dir`: This variable is used for storing the path to the directory where the output files will be saved.
- `csv_output`: CSV file output path
- `html_output`: HTML file output path
- `image_output`: Image file output path

In [ ]:
# Inputs
spreadsheet_url = naas.secret.get("ABI_SPREADSHEET") or "YOUR_GOOGLE_SPREADSHEET_URL"
sheet_name = "CONTENT"
linkedin_url = "https://www.linkedin.com/in/jeremyravenel/"  # EXAMPLE "https://www.linkedin.com/in/XXXXXX/"
title = "Views"  # Chart title
col_date = "PUBLISHED_DATE"
col_value = "VIEWS"  # Column to sum
agg_value = "sum"

# Outputs
output_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "content-engine", date.today().isoformat())
csv_output = os.path.join(output_dir, f"{title.lower()}.csv")
html_output = os.path.join(output_dir, f"{title.lower()}.html")
image_output = os.path.join(output_dir, f"{title.lower()}.png")

## Model

### Get data from Google Sheets spreadsheet

In [ ]:
# Get data
df_posts = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)

# Filter on this week and last week
tw = date.today().strftime("W%W-%Y")
lw = (date.today() - timedelta(days=date.today().weekday() + 7)).strftime("W%W-%Y")
df_posts = df_posts[df_posts["SCENARIO"].isin([tw, lw])]

# Display result
print("Rows:", len(df_posts))
df_posts.head(1)

### Create trend dataframe

In [ ]:
DATE_FORMAT = "%Y-%m-%d"
PERIOD = "%Y-%m-%d"
PERIOD_TEXT = "This day"

def get_trend(
    df_init,
    label,
    col_date,
    col_value,
    agg_value,
    rolling=10
):
    # Init variable
    df = df_init.copy()

    # Groupby period
    if agg_value == "sum":
        df[col_value] = df[col_value].astype(float)
    df[col_date] = pd.to_datetime(df[col_date].str[:-6]).dt.strftime(DATE_FORMAT)
    df = df.groupby(col_date, as_index=False).agg({col_value: agg_value})

    # Rename column
    to_rename = {col_date: "DATE", col_value: "VALUE"}
    df = df.rename(columns=to_rename)

    # Reindex value
    d = datetime.now().date()
    d2 = date.today() - timedelta(days=date.today().weekday() + 7)
    idx = pd.date_range(d2, d, freq="D")
    df.set_index("DATE", drop=True, inplace=True)
    df.index = pd.DatetimeIndex(df.index)
    df = df.reindex(idx, fill_value=0)
    df["DATE"] = pd.DatetimeIndex(df.index)

    # Groupby month
    df["DATE"] = pd.to_datetime(df["DATE"], format=DATE_FORMAT).dt.strftime(PERIOD)
    df = df.groupby("DATE", as_index=False).agg({"VALUE": "sum"})

    # Calc variation
    df.loc[:, "VALUE_COMP"] = 0.
    df.loc[:, "VARV"] = 0.
    df.loc[:, "VARP"] = 1.
    for index, row in df.iterrows():
        if index > 0:
            n = df.loc[df.index[index], "VALUE"]
            n_1 = df.loc[df.index[index - 1], "VALUE"]
            df.loc[df.index[index], "VALUE_COMP"] = n_1
            df.loc[df.index[index], "VARV"] = n - n_1
            if n_1 > 0:
                df.loc[df.index[index], "VARP"] = (n - n_1) / abs(n_1)
    df = df.fillna(0.)

    # Plotly: Date display
    df["DATE_D"] = pd.to_datetime(df["DATE"], format=PERIOD).dt.strftime("%a %d %b")

    # Plotly: Value display
    df["VALUE_D"] = (
        "<b><span style='font-family: Arial;'>"
        + df["VALUE"].map("{:,.0f}".format).str.replace(",", " ")
        + "</span></b>"
    )

    # Plotly: Variation display
    df["VARV_D"] = df["VARV"].map("{:,.0f}".format).str.replace(",", " ")
    df.loc[df["VARV"] >= 0, "VARV_D"] = "+" + df["VARV_D"]
    df["VARP_D"] = df["VARP"].map("{:,.0%}".format).str.replace(",", " ")
    df.loc[df["VARP"] >= 0, "VARP_D"] = "+" + df["VARP_D"]

    # Plotly: hovertext
    df["TEXT"] = (
        "<b><span style='font-size: 14px;'>"
        + df["DATE_D"].astype(str)
        + ": "
        + df["VALUE_D"]
        + "</span></b><br>"
        "<span style='font-size: 12px;'>"
        + df["VARV_D"]
        + " ("
        + df["VARP_D"]
        + ")</span>"
    )
    
    # Add graph title
    df.insert(loc=0, column="SCENARIO", value=pd.to_datetime(df["DATE"]).dt.strftime("W%W-%Y"))
    return df.reset_index(drop=True)


df_trend = get_trend(
    df_posts,
    label=title,
    col_date=col_date,
    col_value=col_value,
    agg_value=agg_value
)
df_trend

### Create title and logo

In [ ]:
# Logo
arrow_up = "https://upload.wikimedia.org/wikipedia/commons/thumb/c/c0/Eo_circle_green_arrow-up.svg/2048px-Eo_circle_green_arrow-up.svg.png"
arrow_down = "https://upload.wikimedia.org/wikipedia/commons/thumb/b/b7/Eo_circle_red_arrow-down.svg/2048px-Eo_circle_red_arrow-down.svg.png"
arrow_right = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/42/Eo_circle_orange_arrow-right.svg/2048px-Eo_circle_orange_arrow-right.svg.png"

# Groupby weeks
df = df_trend.groupby(["SCENARIO"], as_index=False).agg({"VALUE": "sum"})
total = df.loc[df.index[-1], "VALUE"]
total_n1 = df.loc[df.index[0], "VALUE"]
varv = total - total_n1
varp = varv / total_n1

total_d = "{:,.0f}".format(total).replace(",", " ")
varv_d = "{:,.0f}".format(varv).replace(",", " ")
varp_d = "{:,.0%}".format(varp).replace(",", " ")
if varv >= 0:
    varv_d = f"+{varv_d}"
    varp_d = f"+{varp_d}"
title_full = f"<b><span style='font-size: 20px;'>{title}</span></b><br><span style='font-size: 18px;'>{tw}: {total_d} | {varv_d} ({varp_d}) vs last week</span>"

# Logo
logo = None
if varv > 0:
    logo = arrow_up
elif varv > -0.2:
    logo = arrow_right
else:
    logo = arrow_down

### Display linechart

In [ ]:
def create_barchart(
    df,
    title=None,
    logo=None,
    linkedin_url=None,
    label="DATE_D",
    value="VALUE",
    value_d="VALUE_D",
    text="TEXT"
):
    # Init
    fig = go.Figure()

    # Create fig
    fig.add_trace(
        go.Bar(
            x=df[label],
            y=df[value],
            text=df[value_d],
            textposition="outside",
            hoverinfo="text",
            hovertext=df[text],
            marker=dict(color="#1293d2"),
        )
    )
    # Add logo
    fig.add_layout_image(
        dict(
            source=logo,
            xref="paper",
            yref="paper",
            x=0.01,
            y=1.045,
            sizex=0.12,
            sizey=0.12,
            xanchor="right",
            yanchor="bottom",
        )
    )
    # Add annotation
    fig.add_annotation(
        text=f"<i>Source: <a href='{linkedin_url}'>{linkedin_url}</a> / Created at: {date.today().isoformat()}</i>",
        font=dict(family="Arial", color="black"),
        x=0,
        y=-0.25,
        xref="paper",
        yref="paper",
        xanchor="left",
        yanchor="bottom",
        arrowcolor="white",
    )
    fig.update_traces(showlegend=False)
    
    # Update layout
    fig.update_layout(
        title=title,
        title_x=0.09,
        title_font=dict(family="Arial", color="black"),
        paper_bgcolor="#ffffff",
        plot_bgcolor="#ffffff",
        width=1200,
        height=600,
        margin_pad=10,
        margin_b=80
    )
    fig.show()
    return fig

fig = create_barchart(df_trend, title_full, logo, linkedin_url)

## Output

### Save and share your csv file

In [ ]:
# Save your dataframe in CSV
df_trend.to_csv(csv_output, index=False)

### Save and share your graph in HTML


In [ ]:
# Save your graph in HTML
fig.write_html(html_output)

# Share output with naas
# html_link = naas.asset.add(html_output, params={"inline": True})

# -> Uncomment the line below to remove your asset
# naas.asset.delete(html_output)

### Save and share your graph in image


In [ ]:
# Save your graph in PNG
fig.write_image(image_output)

# Share output with naas
image_link = naas.asset.add(image_output, override_prod=True)

# -> Uncomment the line below to remove your asset
# naas.asset.delete(image_output)